## Test BLEU metric

In [2]:
# FUNCTIONS / CLASSES :
import sys
from nltk.tokenize import TweetTokenizer
from torchmetrics import BLEUScore
%load_ext autoreload
%autoreload 2
sys.path.append("./code")
from BLEU import Sentence_BLEU
from data_processing import WMT22

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from torchmetrics.functional import bleu_score

## Toy Example

In [33]:
reference = ["Je me présente je m'appelle Henri"]
good_candidate = "Je me présente je me prénomme Henri"
avg_candidate = "Bonjour mon nom est Henri"
bad_candidate = "J'aime les pizzas"

In [52]:
bleu_score(good_candidate, reference, n_gram = 1)

tensor(0.7143)

In [53]:
bleu_score(avg_candidate, reference, n_gram = 1)

tensor(0.1637)

In [54]:
bleu_score(bad_candidate, reference, n_gram = 1)

tensor(0.)

## Test on WMT Data

In [37]:
df = WMT22.read_data(sl_tl = "ende")

In [38]:
sample = df.sample(1000)

In [39]:
sample

,seg_id,sys,hyp,domain,doc,source,ref,score
1714,1729,bleu_bestmbr,"Dies führt nur dazu, dass Ihr zurückgegebener ...",ecommerce,ecommerce_ecommerce_en_8:English-German,This will only result in your returned item no...,"Dies führt dazu, dass der zurückgegebene Artik...",0.0
28536,396,QUARTZ_TuneReranking,Hier besteht Wehrpflicht für alle Männer im Al...,social,social_t3_rhxge7:English-German,Here military service is mandatory for all men...,Hier ist der Militärdienst obligatorisch für a...,0.0
2611,649,bleurt_bestmbr,"Das Zifferblatt ist in einem sehr guten, wenn ...",ecommerce,ecommerce_ecommerce_en_36:English-German,"The dial is in very good, if not new old stock...","Das Ziffernblatt befindet sich in sehr gutem, ...",0.0
17253,1151,Online-B,Lassen Sie den Netzschalter los.,conversation,conversation_en_fr_CLIENT-02_default_2021-01-0...,Release the power button.,Lassen Sie die Power-Taste los.,-2.0
11571,1527,Lan-Bridge,"Ich habe die Erlaubnis, dieses Konto zu verwen...",social,social_t3_qxpw5l:English-German,I have permission to use this account because ...,"Ich habe die Erlaubnis, dieses Konto zu nutzen...",-6.0
...,...,...,...,...,...,...,...,...
2652,690,bleurt_bestmbr,Bitte beachten Sie: Unsere Büroöffnungszeiten ...,ecommerce,ecommerce_ecommerce_en_24:English-German,Please note: Our office opening times are: Mon...,Bitte beachten Sie: Unsere Bürozeiten sind: Mo...,-0.2
23990,1927,Online-Y,Tippe auf Einstellungen.,conversation,conversation_en_fr_CLIENT-02_default_2020-12-2...,Tap Settings.,Tippen Sie auf „Einstellungen“.,-1.1
2074,56,bleurt_bestmbr,Bei einem so neuen Projekt kann der Vertrauens...,news,news_pr-newswire.16452:English-German,"With such a new project, trust can take time t...","Bei einem so neuen Projekt kann es dauern, bis...",0.0
4182,199,chrf_bestmbr,Edit: Danke an alle für die Ratschläge!,social,social_t3_rpx2w7:English-German,Edit: Thanks everyone for the advice!,Bearbeitung: Vielen Dank an alle für den Tipp!,0.0


In [59]:
df["BLEU"] = df.apply(lambda x : bleu_score(
    preds = x.hyp,
    target = [x.ref],
    n_gram = 1).item(), axis = 1)

In [64]:
df[["score", "BLEU"]].corr(method = "spearman")

,score,BLEU
score,1.000000,0.165826
BLEU,0.165826,1.000000


In [42]:
sample

,seg_id,sys,hyp,domain,doc,source,ref,score,BLEU
1714,1729,bleu_bestmbr,"Dies führt nur dazu, dass Ihr zurückgegebener ...",ecommerce,ecommerce_ecommerce_en_8:English-German,This will only result in your returned item no...,"Dies führt dazu, dass der zurückgegebene Artik...",0.0,0.402988
28536,396,QUARTZ_TuneReranking,Hier besteht Wehrpflicht für alle Männer im Al...,social,social_t3_rhxge7:English-German,Here military service is mandatory for all men...,Hier ist der Militärdienst obligatorisch für a...,0.0,0.000000
2611,649,bleurt_bestmbr,"Das Zifferblatt ist in einem sehr guten, wenn ...",ecommerce,ecommerce_ecommerce_en_36:English-German,"The dial is in very good, if not new old stock...","Das Ziffernblatt befindet sich in sehr gutem, ...",0.0,0.000000
17253,1151,Online-B,Lassen Sie den Netzschalter los.,conversation,conversation_en_fr_CLIENT-02_default_2021-01-0...,Release the power button.,Lassen Sie die Power-Taste los.,-2.0,0.000000
11571,1527,Lan-Bridge,"Ich habe die Erlaubnis, dieses Konto zu verwen...",social,social_t3_qxpw5l:English-German,I have permission to use this account because ...,"Ich habe die Erlaubnis, dieses Konto zu nutzen...",-6.0,0.189672
...,...,...,...,...,...,...,...,...,...
2652,690,bleurt_bestmbr,Bitte beachten Sie: Unsere Büroöffnungszeiten ...,ecommerce,ecommerce_ecommerce_en_24:English-German,Please note: Our office opening times are: Mon...,Bitte beachten Sie: Unsere Bürozeiten sind: Mo...,-0.2,0.341723
23990,1927,Online-Y,Tippe auf Einstellungen.,conversation,conversation_en_fr_CLIENT-02_default_2020-12-2...,Tap Settings.,Tippen Sie auf „Einstellungen“.,-1.1,0.000000
2074,56,bleurt_bestmbr,Bei einem so neuen Projekt kann der Vertrauens...,news,news_pr-newswire.16452:English-German,"With such a new project, trust can take time t...","Bei einem so neuen Projekt kann es dauern, bis...",0.0,0.235785
4182,199,chrf_bestmbr,Edit: Danke an alle für die Ratschläge!,social,social_t3_rpx2w7:English-German,Edit: Thanks everyone for the advice!,Bearbeitung: Vielen Dank an alle für den Tipp!,0.0,0.000000


RuntimeError: KeyboardInterrupt: <EMPTY MESSAGE>